In [1]:
# ML_F1_V5/scripts/01_clean_data_v5.py
"""
PASO 1: Limpieza de datos V5
- Filtra desde 2005 (datos confiables)
- Recalcula puntos con sistema moderno
- Marca abandonos
- Verifica que solo hay carreras (no sprints)
"""

import pandas as pd
import numpy as np
from pathlib import Path

print("\n" + "="*70)
print("PASO 1: LIMPIEZA DE DATOS V5")
print("="*70)

# ==================== CONFIGURACIÓN ====================

INPUT_FILE = '../data/raw/f1_dataset_base.csv'
OUTPUT_FILE = '../data/processed/f1_clean.csv'

# ==================== FUNCIONES ====================

def calcular_puntos_modernos(position):
    """Sistema moderno: P1=25, P2=18, ..., P10=1"""
    puntos_map = {
        1: 25, 2: 18, 3: 15, 4: 12, 5: 10,
        6: 8, 7: 6, 8: 4, 9: 2, 10: 1
    }
    return puntos_map.get(position, 0)

def limpiar_dataset(df):
    """Limpia y prepara el dataset"""
    
    df = df.copy()
    
    # 1. FILTRAR DESDE 2005
    print("\n1. Filtrando temporadas desde 2005...")
    antes = len(df)
    df = df[df['year'] >= 2005].copy()
    print(f"   Antes: {antes} filas")
    print(f"   Después: {len(df)} filas")
    print(f"   Eliminadas: {antes - len(df)} filas (años 2000-2004)")
    
    # 2. CONVERTIR POSICIONES A NUMÉRICO
    print("\n2. Convirtiendo classified_position a numérico...")
    df['position'] = pd.to_numeric(df['classified_position'], errors='coerce')
    df['position'] = df['position'].fillna(999).astype(int)
    print(f"   ✓ Posiciones convertidas")
    print(f"   DNFs marcados como: 999")
    
    # 3. CORREGIR DRIVER_CODE NULOS
    print("\n3. Corrigiendo driver_code nulos...")
    mask_null = df['driver_code'].isnull()
    n_null = mask_null.sum()
    if n_null > 0:
        df.loc[mask_null, 'driver_code'] = 'DRV' + df.loc[mask_null, 'driver_number'].astype(str)
        print(f"   ✓ Corregidos {n_null} códigos nulos")
    else:
        print(f"   ✓ No hay códigos nulos")
    
    # 4. RENOMBRAR COLUMNAS
    df = df.rename(columns={
        'driver_code': 'driver',
        'team_name': 'team'
    })
    
    # 5. RECALCULAR PUNTOS CON SISTEMA MODERNO
    print("\n4. Recalculando puntos con sistema moderno...")
    df['puntos_carrera'] = df['position'].apply(calcular_puntos_modernos)
    
    # Verificar
    ejemplo = df[df['position'] == 1]['puntos_carrera'].iloc[0] if len(df[df['position'] == 1]) > 0 else None
    print(f"   ✓ Sistema moderno aplicado")
    print(f"   Verificación: P1 = {ejemplo} puntos (debe ser 25)")
    
    # 6. MARCAR ABANDONOS
    print("\n5. Marcando abandonos...")
    df['es_abandono'] = df['position'] == 999
    n_abandonos = df['es_abandono'].sum()
    print(f"   ✓ Abandonos identificados: {n_abandonos}")
    print(f"   % abandonos: {n_abandonos/len(df)*100:.1f}%")
    
    # 7. VERIFICAR SPRINTS
    print("\n6. Verificando sprint races...")
    # Contar pilotos por ronda (debería ser ~20, no 40)
    pilotos_por_ronda = df.groupby(['year', 'round']).size()
    max_pilotos = pilotos_por_ronda.max()
    sospechosas = pilotos_por_ronda[pilotos_por_ronda > 25]
    
    if len(sospechosas) > 0:
        print(f"   ⚠️ ADVERTENCIA: {len(sospechosas)} rondas con más de 25 resultados")
        print(f"   Posibles sprints detectadas:")
        print(sospechosas.head())
        print(f"\n   Estas rondas pueden tener sprints incluidas.")
    else:
        print(f"   ✓ No se detectaron sprints")
        print(f"   Máximo pilotos por ronda: {max_pilotos} (normal)")
    
    # 8. SELECCIONAR COLUMNAS FINALES
    columnas_finales = [
        'year', 'round', 'driver', 'team', 
        'position', 'puntos_carrera', 'es_abandono'
    ]
    df = df[columnas_finales]
    
    # 9. ORDENAR
    df = df.sort_values(['year', 'round', 'position']).reset_index(drop=True)
    
    return df

# ==================== ESTADÍSTICAS ====================

def mostrar_estadisticas(df):
    """Muestra estadísticas del dataset limpio"""
    
    print("\n" + "="*70)
    print("ESTADÍSTICAS FINALES")
    print("="*70)
    
    print(f"\nFilas totales: {len(df):,}")
    print(f"Temporadas: {df['year'].min()} - {df['year'].max()}")
    print(f"Pilotos únicos: {df['driver'].nunique()}")
    print(f"Equipos únicos: {df['team'].nunique()}")
    
    print("\n" + "-"*70)
    print("CARRERAS POR TEMPORADA:")
    print("-"*70)
    carreras = df.groupby('year')['round'].max().to_dict()
    for year, n_carreras in carreras.items():
        print(f"  {year}: {n_carreras:2d} carreras")
    
    print("\n" + "-"*70)
    print("TOP 10 PILOTOS (más carreras):")
    print("-"*70)
    top_drivers = df.groupby('driver').size().sort_values(ascending=False).head(10)
    for driver, n_carreras in top_drivers.items():
        print(f"  {driver}: {n_carreras:3d} carreras")
    
    print("\n" + "-"*70)
    print("PRIMERAS 10 FILAS:")
    print("-"*70)
    print(df.head(10).to_string())

# ==================== EJECUCIÓN ====================

if __name__ == "__main__":
    
    # Cargar
    print(f"\nCargando: {INPUT_FILE}")
    df_raw = pd.read_csv(INPUT_FILE)
    print(f"✓ Cargado: {len(df_raw):,} filas")
    
    # Limpiar
    df_clean = limpiar_dataset(df_raw)
    
    # Estadísticas
    mostrar_estadisticas(df_clean)
    
    # Guardar
    print(f"\n{'='*70}")
    print("GUARDANDO DATASET LIMPIO")
    print(f"{'='*70}")
    
    Path(OUTPUT_FILE).parent.mkdir(parents=True, exist_ok=True)
    df_clean.to_csv(OUTPUT_FILE, index=False)
    
    print(f"\n✅ GUARDADO: {OUTPUT_FILE}")
    print(f"   Tamaño: {len(df_clean):,} filas x {len(df_clean.columns)} columnas")
    
    print(f"\n{'='*70}")
    print("✅ PASO 1 COMPLETADO")
    print(f"{'='*70}")
    print("\nSiguiente paso: 02_calculate_context_stats_v5.py")


PASO 1: LIMPIEZA DE DATOS V5

Cargando: ../data/raw/f1_dataset_base.csv
✓ Cargado: 9,985 filas

1. Filtrando temporadas desde 2005...
   Antes: 9985 filas
   Después: 8725 filas
   Eliminadas: 1260 filas (años 2000-2004)

2. Convirtiendo classified_position a numérico...
   ✓ Posiciones convertidas
   DNFs marcados como: 999

3. Corrigiendo driver_code nulos...
   ✓ No hay códigos nulos

4. Recalculando puntos con sistema moderno...
   ✓ Sistema moderno aplicado
   Verificación: P1 = 25 puntos (debe ser 25)

5. Marcando abandonos...
   ✓ Abandonos identificados: 1537
   % abandonos: 17.6%

6. Verificando sprint races...
   ✓ No se detectaron sprints
   Máximo pilotos por ronda: 24 (normal)

ESTADÍSTICAS FINALES

Filas totales: 8,725
Temporadas: 2005 - 2025
Pilotos únicos: 100
Equipos únicos: 38

----------------------------------------------------------------------
CARRERAS POR TEMPORADA:
----------------------------------------------------------------------
  2005: 19 carreras
  2006